In [1]:
# Not needed if pysleep is pip installed (i.e. you probably dont need these lines)
import sys, os
file_dir = os.path.abspath('')
sys.path.insert(0, file_dir+'/../')

In [2]:
#%% Import the tools we need
from mednickdb_pysleep import sleep_features, pysleep_defaults, pysleep_utils, scorefiles 
from mednickdb_pysleep import sleep_architecture, frequency_features, artifact_detection
import pandas as pd
import numpy as np
import yaml
import mne
import warnings
import datetime
warnings.filterwarnings("ignore")

In [3]:
#setup location of files and save locations
edf_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/'
scorefile_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/sleep_scoring/'
edf_filenames = ['Sid1_edf.edf']
scorefile_filenames = ['Sid1_epoch.csv']
ids = ['1']
study_settings_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/study_settings/ExampleStudy2_study_settings.yaml'
save_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/'
nighttime_split_method = 'quartiles' #None or quartiles
spindle_algo = 'Wamsley2012'
do_spindles = False
do_so = False
do_band_power = True


In [8]:
#%% loop through edf's, extract band power, spindles and slow osc (per stage)
edf_filepaths = [edf_base_path + edf_filename for edf_filename in edf_filenames]
scoring_filepaths = [scorefile_base_path + scorefile_filename for scorefile_filename in scorefile_filenames]
features_cont = []
band_power_cont = []

study_settings = yaml.safe_load(open(study_settings_path,'r+'))

for edf_filepath, scorefile_filepath, id_ in zip(edf_filepaths, scoring_filepaths, ids):   
    print('Working on',edf_filepath)
    
    #%%Get stage info
    epoch_stages, epochoffset, starttime = scorefiles.extract_epochstages_from_scorefile(scorefile_filepath, 
                                                                                         study_settings['stage_map'])
    epoch_stages = scorefiles.score_wake_as_waso_wbso_wase(epoch_stages)

    #get the start and end of where we want to extract spindles from (lights off->lights on)
    start_offset, end_offset, \
    _, epoch_stages = sleep_architecture.lights_on_off_and_sleep_latency(epoch_stages,
                                                                         epoch_sync_offset_seconds=epochoffset)
    
    edf = mne.io.read_raw_edf(edf_filepath)
    all_eeg_chans = [v for v in study_settings['known_eeg_chans'].keys() if v in edf.ch_names]

    print(datetime.datetime.now(), '\tDetecting artifacts')
    epochs_with_artifacts = artifact_detection.detect_artifacts(edf_filepath=edf_filepath,
                                                                epochstages=epoch_stages,
                                                                start_offset=start_offset,
                                                                end_offset=end_offset,
                                                                hjorth_threshold=3,
                                                                delta_threshold=5,
                                                                beta_threshold=4,
                                                                chans_to_consider=all_eeg_chans)
    
    print(datetime.datetime.now(), '\t\tRecord contains ', 
          100*len(epochs_with_artifacts)/len(epoch_stages),'% bad epochs that will be ignored')
    
    dropped_epochs = [epoch_stages[i] for i in epochs_with_artifacts]
    if dropped_epochs:
        stages1, dropped_count = np.unique(dropped_epochs, return_counts=True)
        stages2, total_count = np.unique(epoch_stages, return_counts=True)
        for stage in stages1:
            i1 = stages1==stage
            i2 = stages2==stage
            if np.any(i2):   
                percent_for_stage = 100*dropped_count[i1]/total_count[i2]
                print(percent_for_stage,'% of stage',stage, 'were dropped')
    
    if do_band_power:
        print(datetime.datetime.now(), '\tBand power extraction started')
        band_power = frequency_features.extract_band_power(edf_filepath=edf_filepath,
                                                           start_time=start_offset,
                                                           end_time=end_offset,
                                                           chans_to_consider=all_eeg_chans
                                                                if study_settings['chans_for_band_power']=='all'
                                                                else study_settings['chans_for_band_power'],
                                                           epoch_len=pysleep_defaults.band_power_epoch_len)

        band_power_per_epoch = frequency_features.extract_band_power_per_epoch(band_power,
                                                                               epoch_len=pysleep_defaults.epoch_len)


        band_power_w_stage = frequency_features.assign_band_power_stage(band_power_per_epoch, epoch_stages, bad_epochs=epochs_with_artifacts)


        if nighttime_split_method == 'quartiles':
            per_quartile = True
            band_power_w_stage, _ = pysleep_utils.assign_quartiles(band_power_w_stage, epoch_stages)
            groupby = ['quartile', 'stage', 'chan', 'band']
        else:
            per_quartile = False
            groupby = ['stage', 'chan', 'band']

        band_power_w_stage = band_power_w_stage.drop(['onset','duration'], axis=1)
        band_power_w_stage_to_mean = band_power_w_stage.loc[band_power_w_stage['stage'].isin(pysleep_defaults.stages_to_consider), :]
        power_df = band_power_w_stage_to_mean.groupby(groupby).agg(np.nanmean)
        power_df = power_df.drop(['stage_idx','bad_epoch'], axis=1)
        power_df['power'] = pysleep_utils.trunc(power_df['power'], 3)
        band_power_cont.append(power_df)

    if do_spindles or do_so:
        mins_df = sleep_architecture.sleep_stage_architecture(epoch_stages, epochs_to_ignore=epochs_with_artifacts, return_type='dataframe', per_quartile=per_quartile)

        #%% Sleep Features
        features_detected = []

        if do_spindles and 'chans_for_spindles' in study_settings and study_settings['chans_for_spindles']:
            chans_to_consider = all_eeg_chans if study_settings['chans_for_spindles'] == 'all' else study_settings['chans_for_spindles']
            data = sleep_features.load_and_slice_data_for_feature_extraction(edf_filepath=edf_filepath,epochstages=epoch_stages,start_offset=0,end_offset=3000,chans_to_consider=chans_to_consider)
            spindles = sleep_features.detect_spindles(data, start_offset=start_offset, algo=spindle_algo)
            spindles = sleep_features.assign_stage_to_feature_events(spindles, epoch_stages)
            features_detected.append(spindles)

        if do_so and 'chans_for_slow_osc' in study_settings and study_settings['chans_for_slow_osc']:
            chans_to_consider = all_eeg_chans if study_settings['chans_for_slow_osc'] == 'all' else study_settings['chans_for_slow_osc']
            data = sleep_features.load_and_slice_data_for_feature_extraction(edf_filepath=edf_filepath,epochstages=epoch_stages,start_offset=0,end_offset=3000,chans_to_consider=chans_to_consider)
            sos = sleep_features.detect_slow_oscillation(data, start_offset=start_offset)
            sos = sleep_features.assign_stage_to_feature_events(sos, epoch_stages)
            features_detected.append(sos)

        features_df = pd.concat(features_detected, axis=0, sort=False)
        if features_df.shape[0] == 0:
            sleep_feature_data = {}
        else:
            if len(epochstages) > pyparse_defaults.max_nap_len_in_epochs:
                features_df, _ = pysleep_utils.assign_quartiles(features_df, epoch_stages)
                groupby = ['quartile', 'stage', 'chan', 'description']
            else:
                groupby = ['stage', 'chan', 'description']

            features_per_stage = sleep_features.sleep_feature_variables_per_stage(features_df,mins_in_stage_df=mins_df,av_across_channels=False,stages_to_consider=pysleep_defaults.stages_to_consider)

            features_per_stage = features_per_stage.apply(lambda x: pd.to_numeric(x, errors='ignore'))
            features_per_stage = features_per_stage.groupby(groupby).agg(np.nanmean)
            features_per_stage = features_per_stage.apply(lambda x: pysleep_utils.trunc(x,3))
            features_per_stage['id'] = id_
            features_per_stage = features_per_stage.reset_index().set_index('id').reset_index()

        features_cont.append(features_per_stage)
        print(datetime.datetime.now(), '\tSleep Features extraction finished')
    

Working on C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
2019-07-11 02:55:33.711719 	Detecting artifacts
2019-07-11 02:56:25.614715 		Record contains  18.458197611292075 % bad epochs that will be ignored
[20.68965517] % of stage n1 were dropped
[1.00502513] % of stage n2 were dropped
[33.7164751] % of stage n3 were dropped
[3.33333333] % of stage rem were dropped
[75.] % of stage waso were dropped
[95.65217391] % of stage wbso were dropped
2019-07-11 02:56:25.622716 	Band power extraction started


In [10]:
#Combine all spindles into a single dataframe and save to csv
if do_band_power:
    all_band_info = pd.concat(band_power_cont, axis=0)
    all_band_info.to_csv(save_base_path + 'band_power_data.csv', index=False)
if do_spindles or do_so:
    all_feature_info = pd.concat(features_cont, axis=0)
    all_feature_info.to_csv(save_base_path + 'sleep_feature_data.csv', index=False)

In [11]:
all_band_info

power   stage_idx  bad_epoch
quartile stage chan  band                                     
Q1       n1    C3-M2 SWA        143.405  113.500000   0.333333
                     alpha        5.585  113.500000   0.333333
                     beta         0.654  113.500000   0.333333
                     delta       75.878  113.500000   0.333333
                     fastsigma    1.423  113.500000   0.333333
                     sigma        2.261  113.500000   0.333333
                     slowsigma    3.550  113.500000   0.333333
                     theta       23.605  113.500000   0.333333
               C4-M1 SWA        113.283  113.500000   0.333333
                     alpha        4.898  113.500000   0.333333
                     beta         0.656  113.500000   0.333333
                     delta       59.789  113.500000   0.333333
                     fastsigma    1.320  113.500000   0.333333
                     sigma        2.099  113.500000   0.333333
                     slowsigma    3.270  113.500000   0.333333
                     theta       20.682  113.500000   0.333333
               F3-M2 SWA        137.766  113.500000   0.333333
                     alpha        4.520  113.500000   0.333333
                     beta         0.613  113.500000   0.333333
                     delta       77.047  113.500000   0.333333
                     fastsigma    0.941  113.500000   0.333333
                     sigma        1.408  113.500000   0.333333
                     slowsigma    2.018  113.500000   0.333333
                     theta       20.429  113.500000   0.333333
               F4-M1 SWA        142.954  113.500000   0.333333
                     alpha        4.035  113.500000   0.333333
                     beta         0.716  113.500000   0.333333
                     delta       77.945  113.500000   0.333333
                     fastsigma    0.992  113.500000   0.333333
                     sigma        1.434  113.500000   0.333333
...                                 ...         ...        ...
Q4       waso  F3-M2 beta         0.648  757.611111   0.759259
                     delta       60.035  757.611111   0.759259
                     fastsigma    0.830  757.611111   0.759259
                     sigma        1.115  757.611111   0.759259
                     slowsigma    1.457  757.611111   0.759259
                     theta       12.911  757.611111   0.759259
               F4-M1 SWA        119.136  757.611111   0.759259
                     alpha        5.065  757.611111   0.759259
                     beta         0.823  757.611111   0.759259
                     delta       61.427  757.611111   0.759259
                     fastsigma    0.909  757.611111   0.759259
                     sigma        1.191  757.611111   0.759259
                     slowsigma    1.523  757.611111   0.759259
                     theta       12.662  757.611111   0.759259
               O1-M2 SWA        104.361  757.611111   0.759259
                     alpha       47.072  757.611111   0.759259
                     beta         0.953  757.611111   0.759259
                     delta       70.589  757.611111   0.759259
                     fastsigma    1.367  757.611111   0.759259
                     sigma        2.229  757.611111   0.759259
                     slowsigma    3.196  757.611111   0.759259
                     theta       37.724  757.611111   0.759259
               O2-M1 SWA        119.397  757.611111   0.759259
                     alpha       39.963  757.611111   0.759259
                     beta         0.859  757.611111   0.759259
                     delta       72.735  757.611111   0.759259
                     fastsigma    1.332  757.611111   0.759259
                     sigma        1.958  757.611111   0.759259
                     slowsigma    2.692  757.611111   0.759259
                     theta       35.677  757.611111   0.759259

[864 rows x 3 columns]

In [ ]:
all_feature_info